# Импорт необходимых библиотек

In [1]:
import os
import pickle
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings("ignore")

# Загружаем и подготавливаем набор данных

In [2]:
train_df = pd.read_csv('./data/train_data.csv', index_col='match_id_hash')
test_df = pd.read_csv('./data/test_data.csv', index_col='match_id_hash')
train_df

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39675 entries, a400b8f29dece5f4d266f49f1ae2e98a to 9928dfde50efcbdb2055da23dcdbc101
Columns: 615 entries, game_time to radiant_win
dtypes: float64(160), int64(435), object(20)
memory usage: 186.5+ MB


In [4]:
train_df.describe()

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r2_hero_id,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r3_hero_id,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r4_hero_id,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r5_hero_id,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_hero_damage,r5_tower_damage,r5_kill_streaks,r5_multi_kills,r5_life_state_0,r5_life_state_1,r5_life_state_2,r5_healing,r5_slots,d1_hero_id,d

## Обработка hero_id

In [5]:
r_columns_id = [f'r{i}_hero_id' for i in range(1, 6)]
d_columns_id = [f'd{i}_hero_id' for i in range(1, 6)]
r_columns_name = [f'r{i}_hero_name' for i in range(1, 6)]
d_columns_name = [f'd{i}_hero_name' for i in range(1, 6)]
hero_ids_all = []
hero_names_all = []
hero_wins_all = []
for index, row in tqdm(train_df.iterrows()):
    for r in r_columns_id:
        hero_ids_all.append(row[r])
        if row['radiant_win'] == 0:
            hero_wins_all.append(0)
        else:
            hero_wins_all.append(1)
    for d in d_columns_id:
        hero_ids_all.append(row[d])
        if row['radiant_win'] == 0:
            hero_wins_all.append(1)
        else:
            hero_wins_all.append(0)
    for r in r_columns_name:
        hero_names_all.append(row[r])
    for d in d_columns_name:
        hero_names_all.append(row[d])
print(hero_ids_all[:10], hero_names_all[:10], hero_wins_all[:10])

0it [00:00, ?it/s]

[11, 78, 14, 59, 77, 12, 21, 60, 84, 34] ['npc_dota_hero_nevermore', 'npc_dota_hero_brewmaster', 'npc_dota_hero_pudge', 'npc_dota_hero_huskar', 'npc_dota_hero_lycan', 'npc_dota_hero_phantom_lancer', 'npc_dota_hero_windrunner', 'npc_dota_hero_night_stalker', 'npc_dota_hero_ogre_magi', 'npc_dota_hero_tinker'] [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [6]:
hero_ids_unique = list(set(hero_ids_all))
len(hero_ids_unique)

115

In [7]:
heroes_dict = [{'id': i, 'name': '', 'games': 0, 'wins': 0} for i in hero_ids_unique]
for hero in tqdm(heroes_dict):
    for i in range(len(hero_ids_all)):
        if hero_ids_all[i] == hero['id']:
            hero['name'] = hero_names_all[i]
            hero['games'] += 1
            if hero_wins_all[i] == 1:
                hero['wins'] += 1
for hero in heroes_dict:
    hero['win_rate'] = hero['wins'] / hero['games']
heroes_dict

  0%|          | 0/115 [00:00<?, ?it/s]

[{'id': 1,
  'name': 'npc_dota_hero_antimage',
  'games': 3849,
  'wins': 1817,
  'win_rate': 0.4720706677058976},
 {'id': 2,
  'name': 'npc_dota_hero_axe',
  'games': 4033,
  'wins': 2030,
  'win_rate': 0.5033473840813291},
 {'id': 3,
  'name': 'npc_dota_hero_bane',
  'games': 2212,
  'wins': 1146,
  'win_rate': 0.5180831826401446},
 {'id': 4,
  'name': 'npc_dota_hero_bloodseeker',
  'games': 2914,
  'wins': 1500,
  'win_rate': 0.5147563486616334},
 {'id': 5,
  'name': 'npc_dota_hero_crystal_maiden',
  'games': 6584,
  'wins': 3432,
  'win_rate': 0.5212636695018226},
 {'id': 6,
  'name': 'npc_dota_hero_drow_ranger',
  'games': 2530,
  'wins': 1329,
  'win_rate': 0.525296442687747},
 {'id': 7,
  'name': 'npc_dota_hero_earthshaker',
  'games': 3907,
  'wins': 1893,
  'win_rate': 0.48451497312515995},
 {'id': 8,
  'name': 'npc_dota_hero_juggernaut',
  'games': 9102,
  'wins': 4710,
  'win_rate': 0.5174686882003955},
 {'id': 9,
  'name': 'npc_dota_hero_mirana',
  'games': 3979,
  'wins': 

## Обработка предметов

In [8]:
r_columns_id = [f'r{i}_hero_inventory' for i in range(1, 6)]
d_columns_id = [f'd{i}_hero_inventory' for i in range(1, 6)]
item_ids_all = []
item_wins_all = []
for index, row in tqdm(train_df.iterrows()):
    for r in r_columns_id:
        for it in row[r].split(sep=' '):
            item_ids_all.append(it)
            if row['radiant_win'] == 0:
                item_wins_all.append(0)
            else:
                item_wins_all.append(1)
    for d in d_columns_id:
        for it in row[d].split(sep=' '):
            item_ids_all.append(it)
            if row['radiant_win'] == 0:
                item_wins_all.append(1)
            else:
                item_wins_all.append(0)

print(item_ids_all[:10], item_wins_all[:10])

0it [00:00, ?it/s]

['item_tango', 'item_wraith_band', 'item_enchanted_mango', 'item_clarity', 'item_tpscroll', 'no_items', 'item_magic_stick', 'item_wind_lace', 'item_wind_lace', 'item_magic_wand'] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
item_ids_unique = sorted(list(set(item_ids_all)))
len(item_ids_unique)

208

In [10]:
items_dict = [{'id': i, 'games': 0, 'wins': 0} for i in item_ids_unique]
for item in tqdm(items_dict):
    for i in range(len(item_ids_all)):
        if item_ids_all[i] == item['id']:
            item['games'] += 1
            if item_wins_all[i] == 1:
                item['wins'] += 1
for item in items_dict:
    item['win_rate'] = item['wins'] / item['games']
items_dict

  0%|          | 0/208 [00:00<?, ?it/s]

[{'id': 'item_abyssal_blade',
  'games': 2049,
  'wins': 1319,
  'win_rate': 0.6437286481210347},
 {'id': 'item_aegis',
  'games': 2280,
  'wins': 1812,
  'win_rate': 0.7947368421052632},
 {'id': 'item_aeon_disk',
  'games': 1142,
  'wins': 639,
  'win_rate': 0.5595446584938704},
 {'id': 'item_aether_lens',
  'games': 9936,
  'wins': 5470,
  'win_rate': 0.5505233494363929},
 {'id': 'item_ancient_janggo',
  'games': 6876,
  'wins': 3572,
  'win_rate': 0.5194880744618965},
 {'id': 'item_arcane_boots',
  'games': 54336,
  'wins': 27841,
  'win_rate': 0.5123858951707891},
 {'id': 'item_armlet',
  'games': 9482,
  'wins': 4956,
  'win_rate': 0.5226745412360262},
 {'id': 'item_assault',
  'games': 2035,
  'wins': 1272,
  'win_rate': 0.625061425061425},
 {'id': 'item_basher',
  'games': 4429,
  'wins': 2619,
  'win_rate': 0.5913298713027771},
 {'id': 'item_belt_of_strength',
  'games': 1888,
  'wins': 843,
  'win_rate': 0.4465042372881356},
 {'id': 'item_bfury',
  'games': 15581,
  'wins': 84

In [11]:
for item in tqdm(items_dict):
    if item['games'] < 100:
        print(item)

  0%|          | 0/208 [00:00<?, ?it/s]

{'id': 'item_courier', 'games': 2, 'wins': 0, 'win_rate': 0.0}
{'id': 'item_recipe_abyssal_blade', 'games': 4, 'wins': 2, 'win_rate': 0.5}
{'id': 'item_recipe_aeon_disk', 'games': 6, 'wins': 2, 'win_rate': 0.3333333333333333}
{'id': 'item_recipe_aether_lens', 'games': 66, 'wins': 25, 'win_rate': 0.3787878787878788}
{'id': 'item_recipe_ancient_janggo', 'games': 4, 'wins': 0, 'win_rate': 0.0}
{'id': 'item_recipe_armlet', 'games': 7, 'wins': 3, 'win_rate': 0.42857142857142855}
{'id': 'item_recipe_assault', 'games': 2, 'wins': 1, 'win_rate': 0.5}
{'id': 'item_recipe_basher', 'games': 4, 'wins': 1, 'win_rate': 0.25}
{'id': 'item_recipe_black_king_bar', 'games': 9, 'wins': 4, 'win_rate': 0.4444444444444444}
{'id': 'item_recipe_bloodthorn', 'games': 1, 'wins': 1, 'win_rate': 1.0}
{'id': 'item_recipe_bracer', 'games': 2, 'wins': 0, 'win_rate': 0.0}
{'id': 'item_recipe_buckler', 'games': 9, 'wins': 4, 'win_rate': 0.4444444444444444}
{'id': 'item_recipe_crimson_guard', 'games': 1, 'wins': 1, 'wi

In [12]:
train_target = train_df['radiant_win'].values
train_df.drop('radiant_win', axis=1, inplace=True)
train_target

array([0, 1, 1, ..., 0, 1, 1], dtype=int64)

In [13]:
features = list(train_df.columns)
# categorical_features = ['game_mode', 'lobby_type', 'r1_hero_id',  'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id',
#                         'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id', 'r1_randomed',  'r2_randomed',
#                         'r3_randomed', 'r4_randomed', 'r5_randomed', 'd1_randomed', 'd2_randomed', 'd3_randomed', 'd4_randomed',
#                         'd5_randomed', 'r1_pred_vict',  'r2_pred_vict', 'r3_pred_vict', 'r4_pred_vict', 'r5_pred_vict', 'd1_pred_vict',
#                         'd2_pred_vict', 'd3_pred_vict', 'd4_pred_vict', 'd5_pred_vict', 'r1_hero_inventory', 'r2_hero_inventory',
#                         'r3_hero_inventory', 'r4_hero_inventory', 'r5_hero_inventory','d1_hero_inventory', 'd2_hero_inventory',
#                         'd3_hero_inventory', 'd4_hero_inventory', 'd5_hero_inventory']
# numerical_features = list(set(features) - set(categorical_features))
# print('Общее количество признаков: {0}\nКоличество численных признаков: {1}\nКоличество категориальных признаков: {2}\nЧисленные признаки:\n{3}\nКатегориальные признаки:\n{4}'.format(len(features), len(numerical_features), len(categorical_features), numerical_features, categorical_features))

In [21]:
df_temp = pd.read_csv('train_mod_1.csv', index_col='match_id_hash')

# Генерация признаков

In [22]:
def feature_engineering(data: pd.DataFrame, features_old: list):
    df = data.copy()
    features_common = ['game_time', 'radiant_tower_kills', 'dire_tower_kills', 'radiant_barrack_kills', 'dire_barrack_kills',
                       'radiant_aegis', 'dire_aegis', 'radiant_chat', 'dire_chat']
    features_to_drop = list(set(features_old) - set(features_common))
    
    # обрабатываем общие признаки
    df['tower_kills_diff'] = df['radiant_tower_kills'] - df['dire_tower_kills']
    df['barrack_kills_diff'] = df['radiant_barrack_kills'] - df['dire_barrack_kills']
    df['aegis_diff'] = df['radiant_aegis'] - df['dire_aegis']
    df['chat_diff'] = df['radiant_chat'] - df['dire_chat']
#     df['tower_kills_ratio'] = df['radiant_tower_kills'] / df['dire_tower_kills']
#     df['barrack_kills_ratio'] = df['radiant_barrack_kills'] / df['dire_barrack_kills']
#     df['aegis_ratio'] = df['radiant_aegis'] / df['dire_aegis']
#     df['chat_ratio'] = df['radiant_chat'] / df['dire_chat']
#     for index, row in tqdm(df.iterrows()):
#         if df.loc[index]['tower_kills_ratio'] == np.inf or df.loc[index]['tower_kills_ratio'] == -np.inf:
#             df.at[index, 'tower_kills_ratio'] = df.loc[index]['radiant_tower_kills']
#         if df.loc[index]['barrack_kills_ratio'] == np.inf or df.loc[index]['barrack_kills_ratio'] == -np.inf:
#             df.at[index, 'barrack_kills_ratio'] = df.loc[index]['radiant_barrack_kills']
#         if df.loc[index]['aegis_ratio'] == np.inf or df.loc[index]['aegis_ratio'] == -np.inf:
#             df.at[index, 'aegis_ratio'] = df.loc[index]['radiant_aegis']
#         if df.loc[index]['chat_ratio'] == np.inf or df.loc[index]['chat_ratio'] == -np.inf:
#             df.at[index, 'chat_ratio'] = df.loc[index]['radiant_chat']
            
    # расчитываем показатели на команду (sum, mean, std)
    features_player = ['obs_placed', 'sen_placed', 'creeps_stacked', 'camps_stacked', 'rune_pickups', 'firstblood_claimed', 'teamfight_participation', 'towers_killed',
                       'roshans_killed', 'stuns', 'randomed', 'pred_vict', 'gold', 'lh', 'xp', 'x', 'y', 'health', 'max_health', 'max_mana', 'level', 'denies',
                       'nearby_creep_death_count', 'kills', 'deaths', 'assists', 'kda', 'ability_level', 'max_hero_hit', 'gpm_max', 'gpm_avg', 'gpm_delta', 'lhpm_max',
                       'lhpm_avg', 'lhpm_delta', 'dnpm_max', 'dnpm_avg', 'dnpm_delta', 'xppm_max', 'xppm_avg', 'xppm_delta', 'purchases', 'buybacks', 'actions', 'pings',
                       'item_uses', 'ability_uses', 'hero_hits', 'hero_damage', 'tower_damage', 'kill_streaks', 'multi_kills', 'life_state_0', 'life_state_1',
                       'life_state_2', 'healing', 'slots']
    
    features_player = ['obs_placed', 'sen_placed', 'creeps_stacked', 'camps_stacked', 'rune_pickups', 'firstblood_claimed', 'teamfight_participation', 'towers_killed',
                       'roshans_killed', 'stuns', 'randomed', 'pred_vict', 'gold', 'lh', 'xp', 'x', 'y', 'health', 'max_health', 'max_mana', 'level', 'denies',
                       'nearby_creep_death_count', 'kills', 'deaths', 'assists', 'kda', 'ability_level', 'max_hero_hit', 'gpm_delta', 'lhpm_delta', 'dnpm_delta', 
                       'xppm_delta', 'purchases', 'buybacks', 'actions', 'pings', 'item_uses', 'ability_uses', 'hero_hits', 'hero_damage', 'tower_damage', 
                       'kill_streaks', 'multi_kills', 'life_state_0', 'life_state_1', 'life_state_2', 'healing', 'slots']
    
    for c in features_player:
        r_columns = [f'r{i}_{c}' for i in range(1, 6)]
        d_columns = [f'd{i}_{c}' for i in range(1, 6)]

#         df['r_total_' + c] = df[r_columns].sum(1)
#         df['d_total_' + c] = df[d_columns].sum(1)
        
#         df['r_mean_' + c] = df[r_columns].mean(1)
#         df['d_mean_' + c] = df[d_columns].mean(1)
        
#         df['r_std_' + c] = df[r_columns].std(1)
#         df['d_std_' + c] = df[d_columns].std(1)
             
# #         df['total_' + c + '_diff'] = df['r_total_' + c] - df['d_total_' + c]
# #         df['std_' + c + '_diff'] = df['r_std_' + c] - df['d_std_' + c]
# #         df['mean_' + c + '_diff'] = df['r_mean_' + c] - df['d_mean_' + c]
        
#         df['total_' + c + '_ratio'] = df['r_total_' + c] / df['d_total_' + c]
#         df['std_' + c + '_ratio'] = df['r_std_' + c] / df['d_std_' + c]      
#         df['mean_' + c + '_ratio'] = df['r_mean_' + c] / df['d_mean_' + c]
        
#         for index, row in tqdm(df.iterrows()):
#             if df.loc[index]['total_' + c + '_ratio'] == np.inf or df.loc[index]['total_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'total_' + c + '_ratio'] = df.loc[index]['r_total_' + c]
#             if df.loc[index]['std_' + c + '_ratio'] == np.inf or df.loc[index]['std_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'std_' + c + '_ratio'] = df.loc[index]['r_std_' + c]
#             if df.loc[index]['mean_' + c + '_ratio'] == np.inf or df.loc[index]['mean_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'mean_' + c + '_ratio'] = df.loc[index]['r_mean_' + c]
        
        
        #df['r_total_' + c] = df_temp['r_total_' + c]
       # df['d_total_' + c] = df_temp['d_total_' + c]
        
        df['r_mean_' + c] = df_temp['r_mean_' + c]
        df['d_mean_' + c] = df_temp['d_mean_' + c]
        
        #df['r_std_' + c] = df_temp['r_std_' + c]
        #df['d_std_' + c] = df_temp['d_std_' + c]
             
#         df['total_' + c + '_diff'] = df_temp['total_' + c + '_diff']
#         df['std_' + c + '_diff'] = df_temp['std_' + c + '_diff']
#         df['mean_' + c + '_diff'] = df_temp['mean_' + c + '_diff']
        
        #df['total_' + c + '_ratio'] = df_temp['total_' + c + '_ratio']
        #df['std_' + c + '_ratio'] = df_temp['std_' + c + '_ratio']    
        df['mean_' + c + '_ratio'] = df_temp['mean_' + c + '_ratio']
        
#     for c in features_player_:
#         r_columns = [f'r{i}_{c}' for i in range(1, 6)]
#         d_columns = [f'd{i}_{c}' for i in range(1, 6)]
        
#         df['r_total_' + c] = df[r_columns].sum(1)
#         df['d_total_' + c] = df[d_columns].sum(1)
        
#         df['r_mean_' + c] = df[r_columns].mean(1)
#         df['d_mean_' + c] = df[d_columns].mean(1)
        
#         df['r_std_' + c] = df[r_columns].std(1)
#         df['d_std_' + c] = df[d_columns].std(1)
             
# #         df['total_' + c + '_diff'] = df['r_total_' + c] - df['d_total_' + c]
# #         df['std_' + c + '_diff'] = df['r_std_' + c] - df['d_std_' + c]
# #         df['mean_' + c + '_diff'] = df['r_mean_' + c] - df['d_mean_' + c]
        
#         df['total_' + c + '_ratio'] = df['r_total_' + c] / df['d_total_' + c]
#         df['std_' + c + '_ratio'] = df['r_std_' + c] / df['d_std_' + c]      
#         df['mean_' + c + '_ratio'] = df['r_mean_' + c] / df['d_mean_' + c]
        
#         for index, row in tqdm(df.iterrows()):
#             if df.loc[index]['total_' + c + '_ratio'] == np.inf or df.loc[index]['total_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'total_' + c + '_ratio'] = df.loc[index]['r_total_' + c]
#             if df.loc[index]['std_' + c + '_ratio'] == np.inf or df.loc[index]['std_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'std_' + c + '_ratio'] = df.loc[index]['r_std_' + c]
#             if df.loc[index]['mean_' + c + '_ratio'] == np.inf or df.loc[index]['mean_' + c + '_ratio'] == -np.inf:
#                 df.at[index, 'mean_' + c + '_ratio'] = df.loc[index]['r_mean_' + c]
        
    # обрабатываем hero_id
    hero_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
                41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
                78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                112, 113, 114, 119, 120]
    r_h = ['r1_hero_id',  'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id']
    d_h = ['d1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id']
    features_hero_r, features_hero_d = [], []
    for h_id in hero_ids:
        features_hero_r.append('r_hero_id={0}'.format(h_id))
    for h_id in hero_ids:
        features_hero_d.append('d_hero_id={0}'.format(h_id))
    df[features_hero_r] = 0
    df[features_hero_d] = 0
    df['r_heroes_win_rate'] = 0
    df['d_heroes_win_rate'] = 0
    
    for index, row in tqdm(df.iterrows()):
        r_heroes = []
        d_heroes = []
        for r in r_h:
            r_heroes.append(row[r])
        for d in d_h:
            d_heroes.append(row[d])
        for i in hero_ids:
            if i in r_heroes:
                df.at[index, 'r_hero_id={0}'.format(i)] = 1
            if i in d_heroes:
                df.at[index, 'd_hero_id={0}'.format(i)] = 1
        r_win_sum = 0
        d_win_sum = 0
        for hero in heroes_dict:
            if hero['id'] in r_heroes:
                r_win_sum += hero['win_rate']
            if hero['id'] in d_heroes:
                d_win_sum += hero['win_rate']
        df.at[index, 'r_heroes_win_rate'] = r_win_sum / 5
        df.at[index, 'd_heroes_win_rate'] = d_win_sum / 5
    
    df['heroes_win_rate_ratio'] = df['r_heroes_win_rate'] / df['d_heroes_win_rate']
    # df['heroes_win_rate_diff'] = df['r_heroes_win_rate'] - df['d_heroes_win_rate']
    
    # обрабатываем hero_inventory
    item_ids = ['item_abyssal_blade', 'item_aegis', 'item_aeon_disk', 'item_aether_lens', 'item_ancient_janggo', 'item_arcane_boots', 'item_armlet',
                'item_assault', 'item_basher', 'item_belt_of_strength', 'item_bfury', 'item_black_king_bar', 'item_blade_mail', 'item_blade_of_alacrity',
                'item_blades_of_attack', 'item_blight_stone', 'item_blink', 'item_bloodstone', 'item_bloodthorn', 'item_boots', 'item_boots_of_elves',
                'item_bottle', 'item_bracer', 'item_branches', 'item_broadsword', 'item_buckler', 'item_butterfly', 'item_chainmail', 'item_cheese',
                'item_circlet', 'item_clarity', 'item_claymore', 'item_cloak', 'item_courier', 'item_crimson_guard', 'item_cyclone', 'item_dagon',
                'item_dagon_2', 'item_dagon_3', 'item_dagon_4', 'item_dagon_5', 'item_demon_edge', 'item_desolator', 'item_diffusal_blade', 'item_dragon_lance',
                'item_dust', 'item_eagle', 'item_echo_sabre', 'item_enchanted_mango', 'item_energy_booster', 'item_ethereal_blade', 'item_faerie_fire', 'item_flask',
                'item_force_staff', 'item_gauntlets', 'item_gem', 'item_ghost', 'item_glimmer_cape', 'item_gloves', 'item_greater_crit', 'item_guardian_greaves',
                'item_hand_of_midas', 'item_headdress', 'item_heart', 'item_heavens_halberd', 'item_helm_of_iron_will', 'item_helm_of_the_dominator',
                'item_hood_of_defiance', 'item_hurricane_pike', 'item_hyperstone', 'item_infused_raindrop', 'item_invis_sword', 'item_javelin', 'item_kaya',
                'item_lesser_crit', 'item_lifesteal', 'item_lotus_orb', 'item_maelstrom', 'item_magic_stick', 'item_magic_wand', 'item_manta', 'item_mantle',
                'item_mask_of_madness', 'item_medallion_of_courage', 'item_mekansm', 'item_meteor_hammer', 'item_mithril_hammer', 'item_mjollnir',
                'item_monkey_king_bar', 'item_moon_shard', 'item_mystic_staff', 'item_necronomicon', 'item_necronomicon_2', 'item_necronomicon_3',
                'item_null_talisman', 'item_nullifier', 'item_oblivion_staff', 'item_octarine_core', 'item_ogre_axe', 'item_orb_of_venom', 'item_orchid',
                'item_pers', 'item_phase_boots', 'item_pipe', 'item_platemail', 'item_point_booster', 'item_power_treads', 'item_quarterstaff', 'item_quelling_blade',
                'item_radiance', 'item_rapier', 'item_reaver', 'item_recipe_abyssal_blade', 'item_recipe_aeon_disk', 'item_recipe_aether_lens',
                'item_recipe_ancient_janggo', 'item_recipe_armlet', 'item_recipe_assault', 'item_recipe_basher', 'item_recipe_black_king_bar',
                'item_recipe_bloodthorn', 'item_recipe_bracer', 'item_recipe_buckler', 'item_recipe_crimson_guard', 'item_recipe_cyclone', 'item_recipe_dagon',
                'item_recipe_diffusal_blade', 'item_recipe_force_staff', 'item_recipe_greater_crit', 'item_recipe_guardian_greaves', 'item_recipe_hand_of_midas',
                'item_recipe_kaya', 'item_recipe_lesser_crit', 'item_recipe_maelstrom', 'item_recipe_manta', 'item_recipe_mekansm', 'item_recipe_mjollnir',
                'item_recipe_necronomicon', 'item_recipe_null_talisman', 'item_recipe_orchid', 'item_recipe_pipe', 'item_recipe_radiance', 'item_recipe_rod_of_atos',
                'item_recipe_sange', 'item_recipe_shivas_guard', 'item_recipe_silver_edge', 'item_recipe_soul_ring', 'item_recipe_sphere', 'item_recipe_spirit_vessel',
                'item_recipe_travel_boots', 'item_recipe_urn_of_shadows', 'item_recipe_veil_of_discord', 'item_recipe_wraith_band', 'item_recipe_yasha', 'item_refresher',
                'item_refresher_shard', 'item_relic', 'item_ring_of_aquila', 'item_ring_of_basilius', 'item_ring_of_health', 'item_ring_of_protection',
                'item_ring_of_regen', 'item_river_painter', 'item_river_painter3', 'item_river_painter6', 'item_river_painter7', 'item_robe', 'item_rod_of_atos',
                'item_sange', 'item_sange_and_yasha', 'item_satanic', 'item_shadow_amulet', 'item_sheepstick', 'item_shivas_guard', 'item_silver_edge', 'item_skadi',
                'item_slippers', 'item_smoke_of_deceit', 'item_sobi_mask', 'item_solar_crest', 'item_soul_booster', 'item_soul_ring', 'item_sphere', 'item_spirit_vessel',
                'item_staff_of_wizardry', 'item_stout_shield', 'item_talisman_of_evasion', 'item_tango', 'item_tango_single', 'item_tome_of_knowledge', 'item_tpscroll',
                'item_tranquil_boots', 'item_travel_boots', 'item_travel_boots_2', 'item_ultimate_orb', 'item_ultimate_scepter', 'item_urn_of_shadows', 'item_vanguard',
                'item_veil_of_discord', 'item_vitality_booster', 'item_vladmir', 'item_void_stone', 'item_ward_dispenser', 'item_ward_observer', 'item_ward_sentry',
                'item_wind_lace', 'item_wraith_band', 'item_yasha', 'no_items']
    
    item_ids = []
    for item in items_dict:
        if item['games'] >= 100:
            item_ids.append(item['id'])
    r_it = ['r1_hero_inventory',  'r2_hero_inventory', 'r3_hero_inventory', 'r4_hero_inventory', 'r5_hero_inventory']
    d_it = ['d1_hero_inventory', 'd2_hero_inventory', 'd3_hero_inventory', 'd4_hero_inventory', 'd5_hero_inventory']
    features_item_r, features_item_d = [], []
    for it_id in item_ids:
        features_item_r.append('r_item_id={0}'.format(it_id))
    for it_id in item_ids:
        features_item_d.append('d_item_id={0}'.format(it_id))
    df[features_item_r] = 0
    df[features_item_d] = 0
    
    for index, row in tqdm(df.iterrows()):
        r_items = []
        d_items = []
        for r in r_it:
            r_ = row[r].split(sep=' ')
            for it in r_:
                r_items.append(it)
        for d in d_it:
            d_ = row[d].split(sep=' ')
            for it in d_:
                d_items.append(it)
        r_items_c = collections.Counter(r_items)
        d_items_c = collections.Counter(d_items)
        for item in item_ids:
            if item in r_items:
                df.at[index, 'r_item_id={0}'.format(item)] = r_items_c[item]
            if item in d_items:
                df.at[index, 'd_item_id={0}'.format(item)] = d_items_c[item]
        r_wins = []
        d_wins = []
        for it in r_items:
            for item in items_dict:
                if item['id'] == it:
                    r_wins.append(item['win_rate'])
        for it in d_items:
            for item in items_dict:
                if item['id'] == it:
                    d_wins.append(item['win_rate'])
        df.at[index, 'r_items_win_rate'] = np.mean(r_wins)
        df.at[index, 'd_items_win_rate'] = np.mean(d_wins)
    
    df['items_win_rate_ratio'] = df['r_items_win_rate'] / df['d_items_win_rate']
    
    
    
    
    
    df = df.drop(features_to_drop, axis=1)
    #df = df.replace([np.inf, -np.inf], 1)
    df = df.fillna(0)
    return df


In [23]:
train_mod = feature_engineering(data=train_df, features_old=features)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [24]:
train_mod['radiant_win'] = train_target
train_mod

,game_time,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,tower_kills_diff,barrack_kills_diff,aegis_diff,chat_diff,r_mean_obs_placed,d_mean_obs_placed,mean_obs_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio,r_mean_creeps_stacked,d_mean_creeps_stacked,mean_creeps_stacked_ratio,r_mean_camps_stacked,d_mean_camps_stacked,mean_camps_stacked_ratio,r_mean_rune_pickups,d_mean_rune_pickups,mean_rune_pickups_ratio,r_mean_firstblood_claimed,d_mean_firstblood_claimed,mean_firstblood_claimed_ratio,r_mean_teamfight_participation,d_mean_teamfight_participation,mean_teamfight_participation_ratio,r_mean_towers_killed,d_mean_towers_killed,mean_towers_killed_ratio,r_mean_roshans_killed,d_mean_roshans_killed,mean_roshans_killed_ratio,r_mean_stuns,d_mean_stuns,mean_stuns_ratio,r_mean_randomed,d_mean_randomed,mean_randomed_ratio,r_mean_pred_vict,d_mean_pred_vict,mean_pred_vict_ratio,r_mean_gold,d_mean_gold,mean_gold_ratio,r_mean_lh,d_mean_lh,mean_lh_ratio,r_mean_xp,d_mean_xp,mean_xp_ratio,r_mean_x,d_mean_x,mean_x_ratio,r_mean_y,d_mean_y,mean_y_ratio,r_mean_health,d_mean_health,mean_health_ratio,r_mean_max_health,d_mean_max_health,mean_max_health_ratio,r_mean_max_mana,d_mean_max_mana,mean_max_mana_ratio,r_mean_level,d_mean_level,mean_level_ratio,r_mean_denies,d_mean_denies,mean_denies_ratio,r_mean_nearby_creep_death_count,d_mean_nearby_creep_death_count,mean_nearby_creep_death_count_ratio,r_mean_kills,d_mean_kills,mean_kills_ratio,r_mean_deaths,d_mean_deaths,mean_deaths_ratio,r_mean_assists,d_mean_assists,mean_assists_ratio,r_mean_kda,d_mean_kda,mean_kda_ratio,r_mean_ability_level,d_mean_ability_level,mean_ability_level_ratio,r_mean_max_hero_hit,d_mean_max_hero_hit,mean_max_hero_hit_ratio,r_mean_gpm_delta,d_mean_gpm_delta,mean_gpm_delta_ratio,r_mean_lhpm_delta,d_mean_lhpm_delta,mean_lhpm_delta_ratio,r_mean_dnpm_delta,d_mean_dnpm_delta,mean_dnpm_delta_ratio,r_mean_xppm_delta,d_mean_xppm_delta,mean_xppm_delta_ratio,r_mean_purchases,d_mean_purchases,mean_purchases_ratio,r_mean_buybacks,d_mean_buybacks,mean_buybacks_ratio,r_mean_actions,d_mean_actions,mean_actions_ratio,r_mean_pings,d_mean_pings,mean_pings_ratio,r_mean_item_uses,d_mean_item_uses,mean_item_uses_ratio,r_mean_ability_uses,d_mean_ability_uses,mean_ability_uses_ratio,r_mean_hero_hits,d_mean_hero_hits,mean_hero_hits_ratio,r_mean_hero_damage,d_mean_hero_damage,mean_hero_damage_ratio,r_mean_tower_damage,d_mean_tower_damage,mean_tower_damage_ratio,r_mean_kill_streaks,d_mean_kill_streaks,mean_kill_streaks_ratio,r_mean_multi_kills,d_mean_multi_kills,mean_multi_kills_ratio,r_mean_life_state_0,d_mean_life_state_0,mean_life_state_0_ratio,r_mean_life_state_1,d_mean_life_state_1,mean_life_state_1_ratio,r_mean_life_state_2,d_mean_life_state_2,mean_life_state_2_ratio,r_mean_healing,d_mean_healing,mean_healing_ratio,r_mean_slots,d_mean_slots,mean_slots_ratio,r_hero_id=1,r_hero_id=2,r_hero_id=3,r_hero_id=4,r_hero_id=5,r_hero_id=6,r_hero_id=7,r_hero_id=8,r_hero_id=9,r_hero_id=10,r_hero_id=11,r_hero_id=12,r_hero_id=13,r_hero_id=14,r_hero_id=15,r_hero_id=16,r_hero_id=17,r_hero_id=18,r_hero_id=19,r_hero_id=20,r_hero_id=21,r_hero_id=22,r_hero_id=23,r_hero_id=25,r_hero_id=26,r_hero_id=27,r_hero_id=28,r_hero_id=29,r_hero_id=30,r_hero_id=31,r_hero_id=32,r_hero_id=33,r_hero_id=34,r_hero_id=35,r_hero_id=36,r_hero_id=37,r_hero_id=38,r_hero_id=39,r_hero_id=40,r_hero_id=41,r_hero_id=42,r_hero_id=43,r_hero_id=44,r_hero_id=45,r_hero_id=46,r_hero_id=47,r_hero_id=48,r_hero_id=49,r_hero_id=50,r_hero_id=51,r_hero_id=52,r_hero_id=53,r_hero_id=54,r_hero_id=55,r_hero_id=56,r_hero_id=57,r_hero_id=58,r_hero_id=59,r_hero_id=60,r_hero_id=61,r_hero_id=62,r_hero_id=63,r_hero_id=64,r_hero_id=65,r_hero_id=66,r_hero_id=67,r_hero_id=68,r_hero_id=69,r_hero_id=70,r_hero_id=71,r_hero_id=72,r_hero_id=73,r_hero_id=74,r_hero_id=75,r_hero_id=76,r_hero_id=77,r_hero_id=78,r_hero_id=79,r_hero_id=80,r_hero_id=81,r_hero_id=82,r_hero_id=83,r_hero_id

In [25]:
train_mod.to_csv('train_mod.csv')

In [26]:
df_temp = pd.read_csv('test_mod_1.csv', index_col='match_id_hash')
test_mod = feature_engineering(data=test_df, features_old=features)
test_mod

0it [00:00, ?it/s]

0it [00:00, ?it/s]

,game_time,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,tower_kills_diff,barrack_kills_diff,aegis_diff,chat_diff,r_mean_obs_placed,d_mean_obs_placed,mean_obs_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio,r_mean_creeps_stacked,d_mean_creeps_stacked,mean_creeps_stacked_ratio,r_mean_camps_stacked,d_mean_camps_stacked,mean_camps_stacked_ratio,r_mean_rune_pickups,d_mean_rune_pickups,mean_rune_pickups_ratio,r_mean_firstblood_claimed,d_mean_firstblood_claimed,mean_firstblood_claimed_ratio,r_mean_teamfight_participation,d_mean_teamfight_participation,mean_teamfight_participation_ratio,r_mean_towers_killed,d_mean_towers_killed,mean_towers_killed_ratio,r_mean_roshans_killed,d_mean_roshans_killed,mean_roshans_killed_ratio,r_mean_stuns,d_mean_stuns,mean_stuns_ratio,r_mean_randomed,d_mean_randomed,mean_randomed_ratio,r_mean_pred_vict,d_mean_pred_vict,mean_pred_vict_ratio,r_mean_gold,d_mean_gold,mean_gold_ratio,r_mean_lh,d_mean_lh,mean_lh_ratio,r_mean_xp,d_mean_xp,mean_xp_ratio,r_mean_x,d_mean_x,mean_x_ratio,r_mean_y,d_mean_y,mean_y_ratio,r_mean_health,d_mean_health,mean_health_ratio,r_mean_max_health,d_mean_max_health,mean_max_health_ratio,r_mean_max_mana,d_mean_max_mana,mean_max_mana_ratio,r_mean_level,d_mean_level,mean_level_ratio,r_mean_denies,d_mean_denies,mean_denies_ratio,r_mean_nearby_creep_death_count,d_mean_nearby_creep_death_count,mean_nearby_creep_death_count_ratio,r_mean_kills,d_mean_kills,mean_kills_ratio,r_mean_deaths,d_mean_deaths,mean_deaths_ratio,r_mean_assists,d_mean_assists,mean_assists_ratio,r_mean_kda,d_mean_kda,mean_kda_ratio,r_mean_ability_level,d_mean_ability_level,mean_ability_level_ratio,r_mean_max_hero_hit,d_mean_max_hero_hit,mean_max_hero_hit_ratio,r_mean_gpm_delta,d_mean_gpm_delta,mean_gpm_delta_ratio,r_mean_lhpm_delta,d_mean_lhpm_delta,mean_lhpm_delta_ratio,r_mean_dnpm_delta,d_mean_dnpm_delta,mean_dnpm_delta_ratio,r_mean_xppm_delta,d_mean_xppm_delta,mean_xppm_delta_ratio,r_mean_purchases,d_mean_purchases,mean_purchases_ratio,r_mean_buybacks,d_mean_buybacks,mean_buybacks_ratio,r_mean_actions,d_mean_actions,mean_actions_ratio,r_mean_pings,d_mean_pings,mean_pings_ratio,r_mean_item_uses,d_mean_item_uses,mean_item_uses_ratio,r_mean_ability_uses,d_mean_ability_uses,mean_ability_uses_ratio,r_mean_hero_hits,d_mean_hero_hits,mean_hero_hits_ratio,r_mean_hero_damage,d_mean_hero_damage,mean_hero_damage_ratio,r_mean_tower_damage,d_mean_tower_damage,mean_tower_damage_ratio,r_mean_kill_streaks,d_mean_kill_streaks,mean_kill_streaks_ratio,r_mean_multi_kills,d_mean_multi_kills,mean_multi_kills_ratio,r_mean_life_state_0,d_mean_life_state_0,mean_life_state_0_ratio,r_mean_life_state_1,d_mean_life_state_1,mean_life_state_1_ratio,r_mean_life_state_2,d_mean_life_state_2,mean_life_state_2_ratio,r_mean_healing,d_mean_healing,mean_healing_ratio,r_mean_slots,d_mean_slots,mean_slots_ratio,r_hero_id=1,r_hero_id=2,r_hero_id=3,r_hero_id=4,r_hero_id=5,r_hero_id=6,r_hero_id=7,r_hero_id=8,r_hero_id=9,r_hero_id=10,r_hero_id=11,r_hero_id=12,r_hero_id=13,r_hero_id=14,r_hero_id=15,r_hero_id=16,r_hero_id=17,r_hero_id=18,r_hero_id=19,r_hero_id=20,r_hero_id=21,r_hero_id=22,r_hero_id=23,r_hero_id=25,r_hero_id=26,r_hero_id=27,r_hero_id=28,r_hero_id=29,r_hero_id=30,r_hero_id=31,r_hero_id=32,r_hero_id=33,r_hero_id=34,r_hero_id=35,r_hero_id=36,r_hero_id=37,r_hero_id=38,r_hero_id=39,r_hero_id=40,r_hero_id=41,r_hero_id=42,r_hero_id=43,r_hero_id=44,r_hero_id=45,r_hero_id=46,r_hero_id=47,r_hero_id=48,r_hero_id=49,r_hero_id=50,r_hero_id=51,r_hero_id=52,r_hero_id=53,r_hero_id=54,r_hero_id=55,r_hero_id=56,r_hero_id=57,r_hero_id=58,r_hero_id=59,r_hero_id=60,r_hero_id=61,r_hero_id=62,r_hero_id=63,r_hero_id=64,r_hero_id=65,r_hero_id=66,r_hero_id=67,r_hero_id=68,r_hero_id=69,r_hero_id=70,r_hero_id=71,r_hero_id=72,r_hero_id=73,r_hero_id=74,r_hero_id=75,r_hero_id=76,r_hero_id=77,r_hero_id=78,r_hero_id=79,r_hero_id=80,r_hero_id=81,r_hero_id=82,r_hero_id=83,r_hero_id

In [27]:
test_mod.to_csv('test_mod.csv')